## Основные сущности

Ниже перечислены основные сущности, которыми должен оперировать сервер.

### User

Пользователь приложения. Имеет следующие свойства:

* **id** - уникальный идентификатор пользователя
* **username** - уникальное имя пользователя
* **created_at** - время создания пользователя

### Chat

Отдельный чат. Имеет следующие свойства:

* **id** - уникальный идентификатор чата
* **name** - уникальное имя чата
* **users** - список пользователей в чате, отношение многие-ко-многим
* **created_at** - время создания

### Message

Сообщение в чате. Имеет следующие свойства:

* **id** - уникальный идентификатор сообщения
* **chat** - ссылка на идентификатор чата, в который было отправлено сообщение
* **author** - ссылка на идентификатор отправителя сообщения, отношение многие-к-одному
* **text** - текст отправленного сообщения
* **created_at** - время создания

## Основные API методы


### Добавить нового пользователя


Ответ: `id` созданного пользователя.

### Создать новый чат между пользователями

Ответ: `id` созданного чата.

Количество пользователей не ограничено.

### Отправить сообщение в чат от лица пользователя

Ответ: `id` созданного сообщения.

### Получить список чатов конкретного пользователя

Ответ: cписок всех чатов со всеми полями, отсортированный по времени создания последнего сообщения в чате (от позднего к раннему).

### Получить список сообщений в конкретном чате

Ответ: список всех сообщений чата со всеми полями, отсортированный по времени создания сообщения (от раннего к позднему). 

In [1]:
from datetime import datetime

In [2]:
class User:
    def __init__(self, user_id, username):
        self.id = user_id
        self.username = username
        self.created_at = datetime.now()


class Chat:
    def __init__(self, chat_id, name, users):
        self.id = chat_id
        self.name = name
        self.users = users
        self.created_at = datetime.now()


class Message:
    def __init__(self, message_id, chat_id, author_id, text):
        self.id = message_id
        self.chat_id = chat_id
        self.author_id = author_id
        self.text = text
        self.created_at = datetime.now()


class Storage:
    def __init__(self, obj_type):
        self.items = {}
        self.next_item_id = 1
        self.obj_type = obj_type

    def get_next_id(self):
        return self.next_item_id

    def add_item(self, item):
        item.id = self.next_item_id
        self.items[self.next_item_id] = item
        self.next_item_id += 1
        return item.id

    def get_item(self, item_id):
        return self.items.get(item_id)

    def get_all(self):
        return list(self.items.values())

    def remove_item(self, item_id):
        self.items.pop(item_id, None)

    def update_item(self, item_id, item):
        if item_id in self.items:
            self.items[item_id] = item

    def get_items_by_key(self, key, key_value, sort_reversed):
        items = self.items
        filtered_items = []
        for item_id in items:
            item = self.get_item(item_id)
            item_value = getattr(item, key)
            if isinstance(item_value, list):
                if key_value in item_value:
                    filtered_items.append(item)
            elif item_value == key_value:
                filtered_items.append(item)
        return sorted(filtered_items, key=lambda x: x.created_at, reverse=sort_reversed)


class Service:
    def __init__(self, item_type):
        self.storage = Storage()
        self.item_type = item_type

    def validate(self):
        pass

    def get_next_id(self):
        return self.storage.get_next_id()

    def add_item(self, *parameters):
        self.validate(*parameters)
        item_id = self.get_next_id()
        new_item = self.item_type(item_id, *parameters)
        self.storage.add_item(new_item)
        return item_id

    def get_item(self, item_id):
        return self.storage.get_item(item_id)

    def get_items_by_key(self, key, key_value, sort_reversed=False):
        return self.storage.get_items_by_key(key, key_value, sort_reversed)

    def get_all(self):
        return self.storage.get_all()


class UserService(Service):
    def __init__(self):
        self.storage = Storage(User)
        self.item_type = User

    def validate(self, username):
        if not username.strip():
            raise ValueError("Имя пользователя не должно быть пустым")


class ChatService(Service):
    def __init__(self):
        self.storage = Storage(Chat)
        self.item_type = Chat

    def validate(self, name, user_ids):
        if not name.strip():
            raise ValueError("Имя чата не должно быть пустым")
        if not user_ids:
            raise ValueError("список пользователей не должен быть пустым")

class MessageService(Service):
    def __init__(self):
        self.storage = Storage(Message)
        self.item_type = Message

    def validate(self, chat_id, author_id, text):
        if not chat_id:
            raise ValueError("Айди чата не должно быть пустым")
        if not author_id:
            raise ValueError("Айди автора сообщения не должно быть пустым")
        if not text.strip():
            raise ValueError("Сообщение не должно быть пустым")


class ChatApp:
    def __init__(self):
        self.user_service = UserService()
        self.chat_service = ChatService()
        self.message_service = MessageService()

    def add_user(self, username):
        return self.user_service.add_item(username)

    def get_user(self, user_id):
        return self.user_service.get_item(user_id)

    def add_chat(self, name, user_ids):
        return self.chat_service.add_item(name, user_ids)

    def get_chat(self, chat_id):
        return self.chat_service.get_item(chat_id)

    def add_message(self, chat_id, author_id, text):
        return self.message_service.add_item(chat_id, author_id, text)

    def get_message(self, message_id):
        return self.message_service.get_item(message_id)

    def get_messages_by_chat(self, chat_id):
        return self.message_service.get_items_by_key('chat_id', chat_id)

    def get_chats_by_user(self, user_id, sort_reversed=True):
        return self.chat_service.get_items_by_key('users', user_id, sort_reversed)


### Юнит тесты

In [3]:
import unittest

class TestStorage(unittest.TestCase):
    def setUp(self):
        self.storage = Storage(User)
        self.user = User(1, "username")
        self.storage.add_item(self.user)

    def test_add_item(self):
        new_user = User(2, "new_username")
        self.storage.add_item(new_user)
        self.assertEqual(self.storage.get_item(2), new_user)

    def test_get_item(self):
        self.assertEqual(self.storage.get_item(1), self.user)

    def test_get_all(self):
        self.assertEqual(self.storage.get_all(), [self.user])

    def test_remove_item(self):
        self.storage.remove_item(1)
        self.assertIsNone(self.storage.get_item(1))

    def test_update_item(self):
        updated_user = User(1, "updated_username")
        self.storage.update_item(1, updated_user)
        self.assertEqual(self.storage.get_item(1), updated_user)

    def test_get_items_by_key(self):
        new_user = User(2, "new_username")
        self.storage.add_item(new_user)
        filtered_users = self.storage.get_items_by_key("username", "new_username")
        self.assertEqual(filtered_users, [new_user])

class TestUserService(unittest.TestCase):
    def setUp(self):
        self.user_service = UserService()

    def test_validate(self):
        with self.assertRaises(ValueError):
            self.user_service.validate("")

    def test_add_item(self):
        user_id = self.user_service.add_item("username")
        self.assertEqual(user_id, 1)

    def test_get_item(self):
        user_id = self.user_service.add_item("username")
        user = self.user_service.get_item(user_id)
        self.assertEqual(user.id, user_id)
        self.assertEqual(user.username, "username")

    def test_get_items_by_key(self):
        self.user_service.add_item("username")
        filtered_users = self.user_service.get_items_by_key("username", "username")
        self.assertEqual(len(filtered_users), 1)

class TestChatService(unittest.TestCase):
    def setUp(self):
        self.chat_service = ChatService()
        self.user_service = UserService()
        self.user_id = self.user_service.add_item("username")

    def test_validate(self):
        with self.assertRaises(ValueError):
            self.chat_service.validate("", [])

    def test_add_item(self):
        chat_id = self.chat_service.add_item("chatname", [self.user_id])
        self.assertEqual(chat_id, 1)

    def test_get_item(self):
        chat_id = self.chat_service.add_item("chatname", [self.user_id])
        chat = self.chat_service.get_item(chat_id)
        self.assertEqual(chat.id, chat_id)
        self.assertEqual(chat.name, "chatname")

    def test_get_items_by_key(self):
        self.chat_service.add_item("chatname", [self.user_id])
        filtered_chats = self.chat_service.get_items_by_key("name", "chatname")
        self.assertEqual(len(filtered_chats), 1)

class TestMessageService(unittest.TestCase):
    def setUp(self):
        self.message_service = MessageService()
        self.chat_service = ChatService()
        self.user_service = UserService()
        self.user_id = self.user_service.add_item("username")
        self.chat_id = self.chat_service.add_item("chatname", [self.user_id])

    def test_add_item(self):
        message_id = self.message_service.add_item(self.chat_id, self.user_id, "text")
        self.assertEqual(message_id, 1)
        message = self.message_service.get_item(message_id)
        self.assertEqual(message.chat_id, self.chat_id)
        self.assertEqual(message.author_id, self.user_id)
        self.assertEqual(message.text, "text")

    def test_get_items_by_key_author_id(self):
        message_id = self.message_service.add_item(self.chat_id, self.user_id, "text")
        filtered_messages = self.message_service.get_items_by_key("author_id", self.user_id)
        self.assertEqual(len(filtered_messages), 1)
        self.assertEqual(filtered_messages[0].id, message_id)        

        
class TestUser(unittest.TestCase):
    def setUp(self):
        self.user = User(1, "username")

    def test_init(self):
        self.assertEqual(self.user.id, 1)
        self.assertEqual(self.user.username, "username")
        
        
class TestMessage(unittest.TestCase):
    def setUp(self):
        self.message = Message(1, 1, 1, "text")

    def test_init(self):
        self.assertEqual(self.message.id, 1)
        self.assertEqual(self.message.chat_id, 1)
        self.assertEqual(self.message.author_id, 1)
        self.assertEqual(self.message.text, "text")         
        
        
class TestChat(unittest.TestCase):
    def setUp(self):
        self.chat = Chat(1, 'New chat', [1,2])

    def test_init(self):
        self.assertEqual(self.chat.id, 1)
        self.assertEqual(self.chat.name, "New chat")
        self.assertEqual(self.chat.users, [1,2]) 
             

In [4]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_init (__main__.TestChat) ... ok
test_add_item (__main__.TestChatService) ... ok
test_get_item (__main__.TestChatService) ... ok
test_get_items_by_key (__main__.TestChatService) ... ok
test_validate (__main__.TestChatService) ... ok
test_init (__main__.TestMessage) ... ok
test_add_item (__main__.TestMessageService) ... ok
test_get_items_by_key_author_id (__main__.TestMessageService) ... ok
test_add_item (__main__.TestStorage) ... ok
test_get_all (__main__.TestStorage) ... ok
test_get_item (__main__.TestStorage) ... ok
test_get_items_by_key (__main__.TestStorage) ... ERROR
test_remove_item (__main__.TestStorage) ... ok
test_update_item (__main__.TestStorage) ... ok
test_init (__main__.TestUser) ... ok
test_add_item (__main__.TestUserService) ... ok
test_get_item (__main__.TestUserService) ... ok
test_get_items_by_key (__main__.TestUserService) ... ok
test_validate (__main__.TestUserService) ... ok

ERROR: test_get_items_by_key (__main__.TestStorage)
---------------------------------

In [5]:
import time

def main():
    app = ChatApp()

    # добавить нового пользователя
    user_id = app.add_user("Саша")
    print("New user id: ", user_id)
    print()

    # добавить еще одного нового пользователя
    user_id = app.add_user("Маша")
    print("New user id: ", user_id)
    print()
    
    # получить пользователя по id
    user = app.get_user(2)
    print(f'User {2}: {user}')
    print(vars(user))
    print()

    # добавить новый чат
    chat_id = app.add_chat("Чат1", [1, 2])
    print("New chat id: ", chat_id)
    print()
    
    time.sleep(1)

    # добавить новый чат
    chat_id = app.add_chat("Чат2", [1, 2])
    print("New chat id: ", chat_id)
    print()
    
    time.sleep(1)

    # добавить новый чат
    chat_id = app.add_chat("Чат2", [1, 2])
    print("New chat id: ", chat_id)
    print()
    
    # отправить сообщение в чат
    message_id = app.add_message(1, 2, "Сообщение 1")
    print("New message id: ", message_id)
    print()
    
    time.sleep(1)


    # отправить еще одно сообщение в чат
    message_id = app.add_message(1, 1, "Сообщение 2")
    print("New message id: ", message_id)
    print()
    
    time.sleep(1)
    
    # отправить еще одно сообщение в чат
    message_id = app.add_message(1, 2, "Сообщение 3")
    print("New message id: ", message_id)
    print()
    

    # получить список всех сообщений из конкретного чата
    messages = app.get_messages_by_chat(1)
    print("All messages: ", messages)
    for message in messages: 
        print(vars(message))
    print()

    #получить список всех чатов для конкретного пользователя
    chats = app.get_chats_by_user(2)
    print("All chats: ", chats)
    for chat in chats: 
        print(vars(chat))


main()

New user id:  1

New user id:  2

User 2: <__main__.User object at 0x0000027DD2F17370>
{'id': 2, 'username': 'Маша', 'created_at': datetime.datetime(2023, 1, 24, 11, 51, 2, 149431)}

New chat id:  1

New chat id:  2

New chat id:  3

New message id:  1

New message id:  2

New message id:  3

All messages:  [<__main__.Message object at 0x0000027DD2F17520>, <__main__.Message object at 0x0000027DD2F172B0>, <__main__.Message object at 0x0000027DD2F175B0>]
{'id': 1, 'chat_id': 1, 'author_id': 2, 'text': 'Сообщение 1', 'created_at': datetime.datetime(2023, 1, 24, 11, 51, 4, 161185)}
{'id': 2, 'chat_id': 1, 'author_id': 1, 'text': 'Сообщение 2', 'created_at': datetime.datetime(2023, 1, 24, 11, 51, 5, 170789)}
{'id': 3, 'chat_id': 1, 'author_id': 2, 'text': 'Сообщение 3', 'created_at': datetime.datetime(2023, 1, 24, 11, 51, 6, 171415)}

All chats:  [<__main__.Chat object at 0x0000027DD2F176D0>, <__main__.Chat object at 0x0000027DD2F178E0>, <__main__.Chat object at 0x0000027DD2F177C0>]
{'id': 